In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
import json
import re
from openai import OpenAI


In [4]:
with open('description.txt', 'r') as file:
    text = file.read()
def clean_json_data(text):
    """Clean the JSON string and parse it properly"""
    # Remove unicode characters like \u00a0
    cleaned_text = re.sub(r'\\u00a0', '', text)
    # Parse the JSON
    data = json.loads(cleaned_text)
    return data

# Clean the data
products = clean_json_data(text)
print(f"Found {len(products)} products")
print(f"First product model: {products[0]['model_no']}")

def parse_description(desc):
    # Extract short and full descriptions separately
    full_desc_match = re.search(r"Full Description:\s*(.*?)(?:\.|$)", desc)
    short_desc_match = re.search(r"Short Description:\s*(.*?)(?:\.|$)", desc)
    
    # Extract all other key-value style entries
    key_value_pairs = re.findall(r"([\w\s/]+?):\s*(.*?)(?:\.|$)", desc)

    parsed = {
        "full_description": full_desc_match.group(1) if full_desc_match else "",
        # "short_description": short_desc_match.group(1) if short_desc_match else "",
        "specs": {k.strip(): v.strip() for k, v in key_value_pairs 
                  if k not in ["Full Description", "Short Description"]}
    }
    return parsed


# # Apply parsing
# for item in products:
    

def clean_spec_value(key, value):
    value = value.strip()
    
    # Normalize "No XYZ support" → "not supported"
    if re.match(r'no\s+[a-zA-Z0-9\-/ ]+\s+support', value, flags=re.IGNORECASE):
        return "not supported"
    
    # Normalize "XYZ not available" → "not available"
    if re.search(r'not available', value, flags=re.IGNORECASE):
        return "not available"

    return value

def specs_to_string(specs):
    parts = []
    for k, v in specs.items():
        cleaned = clean_spec_value(k, v)
        parts.append(f"{k}: {cleaned}")
    return ". ".join(parts)

def create_embedding_text(item):
    parts = [
        f"Model No: {item['model_no']}",
        # f"Short Description: {item['short_description']}",
        f"Full Description: {item['full_description']}",
        specs_to_string(item["specs"])  # Cleaned specs as one string
    ]
    return ". ".join(parts)

def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)              # Collapse multiple spaces
    text = re.sub(r'\s([:;,.])', r'\1', text)      # Remove space before punctuation
    text = text.strip()
    return text

# Add new field for embedding
for item in products:
    parsed = parse_description(item["description"])
    item["full_description"] = parsed["full_description"]
    # item["short_description"] = parsed["short_description"]
    item["specs"] = parsed["specs"]
    del item["description"]
    specs = item.get("specs", {})
    if "Fi" in specs:
        # Move value to correct key
        specs["Wi-Fi"] = specs.pop("Fi")
    item["embedding_text"] = create_embedding_text(item)
    item["embedding_text"] = item["embedding_text"].lower()
    item["embedding_text"] = normalize_text(item["embedding_text"])


docs = [item["embedding_text"] for item in products]


Found 100 products
First product model: I-7547


In [ ]:
client = OpenAI(api_key="YOUR_OPENAI_API_KEY_HERE")

pc = Pinecone(api_key="YOUR_PINECONE_API_KEY_HERE", environment="us-east-1")

In [8]:
from pinecone import ServerlessSpec
index_name = "model-prompter"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
)
index = pc.Index("model-prompter")

ImportError: cannot import name 'ServerlessSpec' from 'pinecone' (unknown location)

In [9]:
!pip install --upgrade pinecone

In [10]:
!pip show pinecone

Name: pinecone
Version: 7.3.0
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /Users/akshayeiyer/Documents/My Projects/Langchain/VectorDB Pinecone project/venv/lib/python3.10/site-packages
Requires: certifi, pinecone-plugin-assistant, pinecone-plugin-interface, python-dateutil, typing-extensions, urllib3
Required-by: 


In [11]:
from pinecone import Pinecone, ServerlessSpec
index_name = "model-prompter"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",  # or "gcp"
            region="us-west-2"
        )
    )

ImportError: cannot import name 'Pinecone' from 'pinecone' (unknown location)